In [1]:
# data reading and const setting
import pandas as pd
import numpy as np
import random
from enum import Enum
import math
import json

In [2]:
class CrossoverType(Enum):
    PartialMappedCrossover = 1
    OrderCrossover = 2
    PositionBasedCrossover = 3
    
class MutationType(Enum):
    Inversion = 1
    Insertion = 2
    Displacement = 3
    ReciprocalExchange = 4
    
class SelectionType(Enum):
    Deterministic = 1
    Stochastic= 2
    
class GeneticAlgorithm:
    def __init__(self,pop_size,number_of_genes,selection_type,
                 crossover_type,crossover_rate,mutation_type,mutation_rate,
                 compute_objective_value):
        
        self.pop_size = pop_size
        self.selection_type = selection_type
        self.crossover_size = int(pop_size*crossover_rate)
        if(self.crossover_size%2==1):
            self.crossover_size -= 1;
        self.mutation_size =  int(pop_size*mutation_rate)
        self.total_size = self.pop_size+self.mutation_size+self.crossover_size
        self.number_of_genes = number_of_genes
        self.crossover_type = crossover_type
        self.crossover_rate = crossover_rate
        self.mutation_type = mutation_type
        self.mutation_rate = mutation_rate
        self.compute_objective_value = compute_objective_value
        self.least_fitness_factor = 0.3
        self.mapping = [-1 for i in range(self.number_of_genes)] #for crossover
        
    def initialize(self):
        self.selected_chromosomes = np.zeros((self.pop_size,self.number_of_genes))
        self.indexs = np.arange(self.total_size)
        self.chromosomes = np.zeros((self.total_size,self.number_of_genes),dtype=int)
#         self.chromosomes = np.array([[range(1, self.number_of_genes + 1)] * self.total_size], dtype=int)
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):  
                self.chromosomes[i][j] = j
            np.random.shuffle(self.chromosomes[i])
       
        for i in range(self.pop_size,self.total_size):
            for j in range(self.number_of_genes):
                self.chromosomes[i][j] = -1
#                 self.chromosomes[i][j] = j
                
        self.fitness = np.zeros(self.total_size) 
        self.objective_values = np.zeros(self.total_size)
        self.best_chromosome = np.zeros(self.number_of_genes,dtype=int)
        self.best_fitness = 0
        
    def evaluate_fitness(self):
        for i,chromosome in enumerate(self.chromosomes[:self.pop_size]):
            self.objective_values[i] = self.compute_objective_value(chromosome)
           
        min_obj_val = np.min(self.objective_values)
        max_obj_val = np.max(self.objective_values)
        range_obj_val = max_obj_val-min_obj_val
        
        for i,obj in enumerate(self.objective_values):
            self.fitness[i] = max(self.least_fitness_factor*range_obj_val,pow(10,-5))+\
                (max_obj_val-obj)
                
    def update_best_solution(self):
        best_index = np.argmax(self.fitness)
        if(self.best_fitness<self.fitness[best_index]):
            self.best_fitness = self.fitness[best_index]
            for i,gene in enumerate(self.chromosomes[best_index]):
                self.best_chromosome[i] = gene
    
    def shuffle_index(self,length):
        for i in range(length):
            self.indexs[i] = i
        np.random.shuffle(self.indexs[:length])
            
    def perform_crossover_operation(self):
        self.shuffle_index(self.pop_size)
        
        child1_index = self.pop_size
        child2_index = self.pop_size+1
        count_of_crossover = int(self.crossover_size/2)
        for i in range(count_of_crossover):
            parent1_index = self.indexs[i]
            parent2_index  = self.indexs[i+1]
            
            if(self.crossover_type == CrossoverType.PartialMappedCrossover):
                self.partial_mapped_crossover(parent1_index,parent2_index,child1_index,child2_index)
                self.objective_values[child1_index] = self.compute_objective_value(self.chromosomes[child1_index])
                self.objective_values[child2_index] = self.compute_objective_value(self.chromosomes[child2_index])
            
            child1_index +=2
            child2_index +=2
        
    def partial_mapped_crossover(self,p1,p2,c1,c2):
        #reset
        for i in range(self.number_of_genes):
            self.mapping[i] = -1
         
        rand1 = random.randint(0,self.number_of_genes-2)
        rand2 = random.randint(rand1+1,self.number_of_genes-1)
       
        for i in range(rand1,rand2+1):
            c1_gene = self.chromosomes[p2][i] 
            c2_gene = self.chromosomes[p1][i]
            
            if(c1_gene==c2_gene):
                continue
            
            elif(self.mapping[c1_gene]==-1 and self.mapping[c2_gene]==-1):
                self.mapping[c1_gene] = c2_gene
                self.mapping[c2_gene] = c1_gene
                
            elif(self.mapping[c1_gene]==-1):
                self.mapping[c1_gene] =  self.mapping[c2_gene]
                self.mapping[self.mapping[c2_gene]] = c1_gene
                self.mapping[c2_gene] = -2
                
            elif (self.mapping[c2_gene]==-1):
                self.mapping[c2_gene] =  self.mapping[c1_gene]
                self.mapping[self.mapping[c1_gene]] = c2_gene
                self.mapping[c1_gene] = -2
                
            else:
                self.mapping[self.mapping[c1_gene]] = self.mapping[c2_gene]
                self.mapping[self.mapping[c2_gene]] = self.mapping[c1_gene]
                self.mapping[c1_gene] = -3
                self.mapping[c2_gene] = -3
                
        for i in range(self.number_of_genes):
            if(i>=rand1 and i<=rand2):
                self.chromosomes[c1][i] =  self.chromosomes[p2][i]
                self.chromosomes[c2][i] =  self.chromosomes[p1][i]
            else:
                if(self.mapping[self.chromosomes[p1][i]] >=0):
                    self.chromosomes[c1][i] = self.mapping[self.chromosomes[p1][i]]
                else:
                    self.chromosomes[c1][i] =self.chromosomes[p1][i]        
                
                if(self.mapping[self.chromosomes[p2][i]] >=0):
                    self.chromosomes[c2][i] = self.mapping[self.chromosomes[p2][i]]
                else:
                    self.chromosomes[c2][i] =self.chromosomes[p2][i]
        
    def do_roulette_wheel_selection(self,fitness_list):
        sum_fitness = sum(fitness_list)
        transition_probability = [fitness/sum_fitness for fitness in fitness_list]
        
        rand = random.random()
        sum_prob = 0
        for i,prob in enumerate(transition_probability):
            sum_prob += prob
            if(sum_prob>=rand):
               return i
        
    def perform_selection(self):
        if self.selection_type == SelectionType.Deterministic:
            index = np.argsort(self.fitness)[::-1]
        
        elif self.selection_type == SelectionType.Stochastic:
            index = [self.do_roulette_wheel_selection(self.fitness) for i in range(self.pop_size)]
        
        else:
            index = self.shuffle_index(self.total_size)
        
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):
                self.selected_chromosomes[i][j] =  self.chromosomes[index[i]][j]
        
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):
                self.chromosomes[i][j] = self.selected_chromosomes[i][j]
                
    def perform_mutation_operation(self):
        self.shuffle_index(self.pop_size+self.crossover_size)
        child1_index = self.pop_size+self.crossover_size
        for i in range(self.mutation_size):
            if(self.mutation_type==MutationType.Inversion):
                parent1_index = self.indexs[i]
                self.inversion_mutation(parent1_index,child1_index)
                self.objective_values[child1_index] = self.compute_objective_value(self.chromosomes[child1_index])
                child1_index += 1
            
    def inversion_mutation(self,p1,c1):
        rand1 = random.randint(0,self.number_of_genes-2)
        rand2 = random.randint(rand1+1,self.number_of_genes-1)
        for i in range(self.number_of_genes):
            if(i<rand1 or i>rand2):
                self.chromosomes[c1][i] = self.chromosomes[p1][i]
            else:
                index = rand2-(i-rand1)
                self.chromosomes[c1][i] = self.chromosomes[p1][index]

In [3]:
#old version, revise data when reading
def read_from_excel(filename, n):
    firefighter_df = pd.read_excel("data/" + filename + "_firefighter_route.xlsx")
    fire_df = pd.read_excel("data/" + filename + "_fire_route.xlsx") 
    burning_time_df = pd.read_excel("data/" + filename + "_burning_time.xlsx") 
    processing_time_df = pd.read_excel("data/" + filename + "_processing_time.xlsx") 
    A_p = [[-1 for i in range(n+1)] for j in range(n+1)]
    A_f = [[0 for i in range(n+1)] for j in range(n+1)]
    processing_time = {}
    for i in range(len(fire_df.index)):
        A_f[fire_df.iloc[i]['j']][fire_df.iloc[i]['i']] = fire_df.iloc[i]['travel time']
    for i in range(0, n+1):
        for j in range(0, n+1):
            if A_f[i][j] != 0:
                A_f[i][j] += burning_time_df.iloc[i-1]['burning time']
            elif i!=j:
                A_f[i][j] = -1
    for i in range(len(firefighter_df.index)):
        A_p[firefighter_df.iloc[i]['j']][firefighter_df.iloc[i]['i']] = firefighter_df.iloc[i]['travel time']
    for i in range(n+1):
#         A_p[0][i] = 0
        A_p[i][0] = 0
    for i in range(n):
        processing_time[i+1] = processing_time_df.iloc[i]['processing time']
    processing_time[0] = 10000
    data = {'A_p': A_p, 'A_f': A_f, 'processing_time': processing_time, 'n': n}
    return data    

In [46]:
#new version, only return raw data
def read_from_excel(filename, n):
    node_df = pd.read_excel("data/"+filename+"_nodeInformation.xlsx")
    firefighter_df = pd.read_excel("data/"+filename+"_firefighter_route.xlsx")
    fire_df = pd.read_excel("data/"+filename+"_fire_route.xlsx")
    data = {}
    data['n'] = n
    data['NODE_POS'] = dict([(i+1, [node_df.iloc[i]['x'], node_df.iloc[i]['y']]) for i in range(len(node_df.index))])
    data['q'] = dict([(i+1, node_df.iloc[i]['quantity']) for i in range(len(node_df.index))])
    data['b'] = dict([(i+1, node_df.iloc[i]['value']) for i in range(len(node_df.index))])
    data['h'] = dict([(i+1, node_df.iloc[i]['burning time']) for i in range(len(node_df.index))])
    data['A_p'], data['A_f'] = [[-1 for i in range(n)]for j in range(n)], [[0 for i in range(n)]for j in range(n)]
    data['A_p_list'], data['A_f_list'] = [], []
    data['lamb'] = {}
    data['tau'] = {}
    for i in fire_df.iloc:
        data['A_f_list'].append((int(i['i']), int(i['j'])))
        data['A_f'][int(i['i']-1)][int(i['j']-1)] = i['travel time']
        data['lamb'][(int(i['i']), int(i['j']))] = i['travel time']
    for i in firefighter_df.iloc:
        data['A_p_list'].append((int(i['i']), int(i['j'])))
        data['A_p'][int(i['i']-1)][int(i['j']-1)] = i['travel time']
        data['tau'][(int(i['i']), int(i['j']), 1)] = i['travel time']
    return data

In [58]:
class FFProblem:
    def __init__(self,data, N_D, N_F, T, P):
        self.M = 9999
        self.P = P
        self.n = data['n']
        self.A_p = np.array(data['A_p'])
        self.A_p = np.vstack((np.array([-1]*int(data['n'])), self.A_p))
        self.A_p = np.hstack((np.array([0]*int(data['n']+1)).reshape(int(data['n']+1), 1), self.A_p))
        self.A_f = np.array(data['A_f']) 
        self.A_f = np.vstack((np.array([0]*int(data['n'])), self.A_f))
        self.A_f = np.hstack((np.array([0]*int(data['n']+1)).reshape(int(data['n']+1), 1), self.A_f))
        for i in range(1, self.n+1):
            for j in range(self.n+1):
                if self.A_f[i][j] != 0:
                    self.A_f[i][j] += data['h'][i]
                elif i!=j:
                    self.A_f[i][j] = -1

        self.N_D = N_D
        self.N_F = N_F
        self.processing_time = dict([(i, math.ceil(data['b'][i]*data['h'][i]/self.P[1])) for i in range(1, data['n']+1)])
        self.processing_time[0] = self.M
        self.T = T
        self.b = [ data['b'][i] for i in range(1, self.n+1)]
        self.b.insert(0, 0)
    
    def compute_objective_value(self,po):
        protect_order = list(po)
        for i in self.N_D:
            protect_order.insert(0, i)
        protected = [(0, 1)[i in self.N_D] for i in range(self.n+1)]
        t = 0
        
        for i in range(protect_order.index(0)):            
            st, dt = protect_order[i], protect_order[i+1]
            bt = self.get_breakout_time(self.N_F[0], protected)
            t += self.processing_time[st]
            t += self.move_to_next_node(st, dt, t, bt)

            if t <= self.T:
                protected[st] = 1
            else:
                return self.M

        burned = list(map(lambda i: int(i < self.T), self.get_breakout_time(self.N_F[0], protected)))
        return np.sum(np.transpose(np.array(burned)).dot(self.b))
        
    def get_breakout_time(self, fire_source, protected):
        unvisit = [*range(self.n+1)]
        cost = [(self.M, 0)[i==fire_source] for i in range(self.n+1)]
        path = [-1] * (self.n+1)
        cur = fire_source
        
        while 1:
            for idx, val in enumerate(list(self.A_f[cur, :])):
                if cost[cur] + val < cost[idx] and protected[idx] != 1 and val != -1:
                    path[idx] = cur
                    cost[idx] = cost[cur] + val
            unvisit.remove(cur)
            if len(unvisit) == 0: break
            cur = min([(cost[unvisit[i]], unvisit[i]) for i in range(len(unvisit))])[1]
                
        return cost
    
    def move_to_next_node(self, st, dt, t, breakout_time): #st, dt are 1-based
        unvisit = [i for i in range(self.n+1)]
        cost = [self.M for i in range(self.n+1)]
        cost[st] = 0
        path = [-1] * (self.n+1)
        cur = st
        while len(unvisit) != 0:
            for i, val in enumerate(list(self.A_p[cur, :])):
                if cost[cur] + val < cost[i] and t + cost[cur] + val <= breakout_time[i] and val >= 0:
                    path[i] = cur
                    cost[i] = cost[cur] + val
                
            unvisit.remove(cur)
            if len(unvisit) == 0: break
            temp = []
            for idx in unvisit:
                temp.append((cost[idx], idx))
            cur = min(temp)[1]
        return cost[dt]
        
    def get_route_information(self, chromosome):
        chromosome = list(chromosome[:chromosome.index(0)])
        chromosome.insert(0, self.N_D[0])
        x, u, v, u_bar, v_bar= {}, {}, {}, {}, {}
        for i in range(1, self.n+1):
            for t in range(self.T+1):
                u[(i, t)], v[(i, t)], v_bar[(i, t)] = 0, 0, 0
                for k in range(1, 2):
                    u_bar[(i, k, t)] = 0
                    for j in range(1, self.n+1):
                        x[(i, j, k, t)] = 0
        for nd in N_D:
            for t in range(self.T+1):
                v_bar[(nd, t)] = 1
                        
        protected = [(0, 1)[i in self.N_D] for i in range(self.n+1)]
        t = 0
        
        for i in range(len(chromosome)-1):
            st, dt = chromosome[i], chromosome[i+1]
            bt = self.get_breakout_time(self.N_F[0], protected)
            
            unvisit = [i for i in range(self.n+1)]
            cost = [self.M for i in range(self.n+1)]
            cost[st] = 0
            path = [-1] * (self.n+1)
            cur = st
            while 1:
                for i, val in enumerate(list(self.A_p[cur, :])):
                    if cost[cur] + val < cost[i] and t + cost[cur] + val <= bt[i] and val >= 0:
                        path[i] = cur
                        cost[i] = cost[cur] + val

                unvisit.remove(cur)
                if len(unvisit) <= 0: break
                cur = min([(cost[unvisit[i]], unvisit[i]) for i in range(len(unvisit))])[1]
            
            cur = dt
            while 1:
                x[(cur, path[cur], 1, t+cost[cur]-cost[path[cur]])] = 1
                if path[cur] == st: break
                cur = path[cur]
            x[(st, cur, 1, t)] = 1
            t += int(cost[dt])
            x[(dt, dt, 1, t)] = 1
            u_bar[(dt, 1, t)] = 1
            protected[dt] = 1
            for t in range(t+1, self.T+1):
                v_bar[(dt, t)] = 1
            t += self.processing_time[dt]
            
        bt = self.get_breakout_time(self.N_F[0], protected)
        for i in range(1, self.n+1):
            for t in range(0, self.T+1):
                if t == bt[i]: u[(i, t)] = 1
                elif t > bt[i]: v[(i, t)] = 1
        
        return x, u, v, u_bar, v_bar
            
        
    
    def convert_to_json(self, data, chromosome):
        result = {}
        result['N'] = [*range(1, self.n+1)]
        result['NODE_POS'] = dict([(i, [int(data['NODE_POS'][i][0]), int(data['NODE_POS'][i][1])]) for i in data['NODE_POS']])
        result['N_D'] = self.N_D
        result['N_F'] = self.N_F
        result['K'] = [1]
        result['T'] = [*range(1, self.T+1)]
        result['A_p'] = [str(i) for i in data['A_p_list']]
        result['A_f'] = [str(i) for i in data['A_f_list']]
        result['tau'] = dict([(str(i), int(data['tau'][i])) for i in data['tau']])
        result['lamb'] = dict([(str(i), int(data['lamb'][i])) for i in data['lamb']])
        result['q'] = dict([(i, int(data['q'][i])) for i in data['q']])
        result['b'] = dict([(i, int(data['b'][i])) for i in data['b']])
        result['h'] = dict([(i, int(data['h'][i])) for i in data['h']])
        x, u, v, u_bar, v_bar = self.get_route_information(chromosome)
        result['x'] = dict([(str(i), x[i]) for i in x])
        result['u'] = dict([(str(i), u[i]) for i in u])
        result['v'] = dict([(str(i), v[i]) for i in v])
        result['u_bar'] = dict([(str(i), u_bar[i]) for i in u_bar])
        result['v_bar'] = dict([(str(i), v_bar[i]) for i in v_bar])

        return result
        
        

In [59]:
N_D = [30] # 1 base
N_F = [1] # 1 base
n = 30
filename = "G30"
data = read_from_excel(filename, n)
T = 100
P = {1:2}
ff = FFProblem(data, N_D, N_F, T, P)

po = [15, 0, 13, 26, 29, 27, 11, 10, 20, 25,  6,  1, 19,  20, 28, 21, 14, 12, 17,  2, 23,  5,  7,  3,
 24,  4,  8, 18, 16,  9]
print(ff.compute_objective_value(po))

# print(ff.convert_to_json(data, po))
json_data = json.dumps(ff.convert_to_json(data, po))
with open(filename+"_data.json", 'w') as file:
    file.write(json.dumps(ff.convert_to_json(data, po)))

85
x( 15 30 1 24.0 )=1
x( 30 15 1 0 )=1
x( 15 30 1 24.0 )=1
x( 30 15 1 0 )=1


In [10]:
N_D = [30] # 1 base
N_F = [1] # 1 base
n = 30
data = read_from_excel("G30", n)
T = 100
P = {1:2}
ff = FFProblem(data, N_D, N_F, T, P)

pop_size = 70
# pop_size = 20
selection_type = SelectionType.Deterministic
crossover_type = CrossoverType.PartialMappedCrossover
crossover_rate = 0.2
mutation_type = MutationType.Inversion
mutation_rate = 0.1
solver = GeneticAlgorithm(pop_size,ff.n ,selection_type,
                          crossover_type,crossover_rate,
                          mutation_type,mutation_rate,
                          ff.compute_objective_value)
solver.initialize()
# print(solver.chromosomes)
# print(f"{solver.best_chromosome}")

for i in range(100):
    solver.perform_crossover_operation()
    solver.perform_mutation_operation()
    solver.evaluate_fitness()
    solver.update_best_solution()
    solver.perform_selection()
    if(i%10==0):
        print(F"iteration {i} :")
        print(f"{solver.best_chromosome}: {ff.compute_objective_value(solver.best_chromosome)}")
        
# print(ff.compute_objective_value([0, 15, 13, 26, 29, 27, 11, 10, 20, 25,  6,  1, 19,  20, 28, 21, 14, 12, 17,  2, 23,  5,  7,  3,
#  24,  4,  8, 18, 16,  9]))

iteration 0 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 21]: 85
iteration 10 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 21]: 85
iteration 20 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 21]: 85
iteration 30 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 21]: 85
iteration 40 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 21]: 85
iteration 50 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 21]: 85
iteration 60 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 21]: 85
iteration 70 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 21]: 85
iteration 80 :
[15  0  3 26  2  9  7 18 25 24 17 16  8  4 20 12 28  1  5 27  6 13 11 29
 10 23 22 19 14 2